In [1]:
import tensorflow as tf
import numpy as np
import gym
from go_ai import data, metrics, policies
from go_ai.models import value_model
import matplotlib.pyplot as plt
import shutil
import multiprocessing as mp

# Hyperparameters

In [2]:
BOARD_SIZE = 9

In [25]:
ITERATIONS = 256
EPISODES_PER_ITERATION = 128
NUM_EVAL_GAMES = 32

In [4]:
EPISODES_DIR = './data/'

In [5]:
BATCH_SIZE = 32
LEARNING_RATE = 2e-3

In [6]:
WEIGHTS_DIR = 'model_weights/'
CHECKPOINT_PATH = WEIGHTS_DIR + 'checkpoint_{}x{}.h5'.format(BOARD_SIZE, BOARD_SIZE)
TMP_WEIGHTS_PATH = WEIGHTS_DIR + 'tmp.h5'
LOAD_SAVED_MODELS = False

In [7]:
NUM_WORKERS = mp.cpu_count()

# Go Environment
Train on a small board for fast training and efficient debugging

In [8]:
go_env = gym.make('gym_go:go-v0', size=BOARD_SIZE)

# Metrics and Tensorboard

In [9]:
DEMO_TRAJECTORY_PATH = 'logs/a_trajectory.png'

Metrics

In [10]:
tb_metrics = {}
for metric_key in ['val_loss', 'move_loss']:
    tb_metrics[metric_key] = tf.keras.metrics.Mean('{}'.format(metric_key), 
                                                   dtype=tf.float32)
tb_metrics['pred_win_acc'] = tf.keras.metrics.Accuracy()

Tensorboard

# Machine Learning Models

In [11]:
val_net = value_model.make_val_net(BOARD_SIZE)

In [12]:
_ = tf.keras.utils.plot_model(val_net, to_file='logs/model.png')

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [13]:
val_net.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 9, 9, 128)         7040      
_________________________________________________________________
batch_normalization (BatchNo (None, 9, 9, 128)         512       
_________________________________________________________________
re_lu (ReLU)                 (None, 9, 9, 128)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 128)         147584    
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 9, 128)         512       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 9, 9, 128)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          7

In [14]:
if LOAD_SAVED_MODELS:
    val_net.load_weights(CHECKPOINT_PATH)
    print("Starting from checkpoint")
else:
    val_net.save_weights(CHECKPOINT_PATH)
    print("Initialized checkpoint and temp")
    
# Sync temp with checkpoint
val_net.save_weights(TMP_WEIGHTS_PATH)

Initialized checkpoint and temp


# Policies

In [15]:
temp_policy_args = {
    'mode': 'values',
    'board_size': BOARD_SIZE,
    'model_path': TMP_WEIGHTS_PATH,
}

In [16]:
checkpoint_policy_args = {
    'mode': 'values',
    'board_size': BOARD_SIZE,
    'model_path': CHECKPOINT_PATH,
}

In [17]:
random_policy_args = {
    'mode': 'random',
    'board_size': BOARD_SIZE,
}

In [18]:
greedy_policy_args = {
    'mode': 'greedy',
    'board_size': BOARD_SIZE,
}

# Demo and Time Games

Symmetries

In [19]:
%%time
go_env.reset()
action = (1, 2)
next_state, _, _, _ = go_env.step(action)
metrics.plot_symmetries(next_state, 'logs/symmetries.jpg')

CPU times: user 125 ms, sys: 6.69 ms, total: 132 ms
Wall time: 131 ms


With replay memory

In [21]:
%%time
data.make_episodes(temp_policy_args, temp_policy_args, 1, num_workers=1, 
                   outdir=EPISODES_DIR)

Episode worker: 1it [00:17, 17.57s/it]
values vs. values: 100%|██████████| 1/1 [00:00<00:00, 984.58it/s, 100.0% WIN]

CPU times: user 21.7 s, sys: 2.43 s, total: 24.1 s
Wall time: 17.8 s


1.0

# Train

In [26]:
for iteration in range(ITERATIONS):
    # Optimization
    
    # Make and write out the episode data
    data.make_episodes(temp_policy_args, temp_policy_args, EPISODES_PER_ITERATION, 
                       num_workers=NUM_WORKERS, outdir=EPISODES_DIR)
    # Read in the episode data
    np_data = data.episodes_from_dir(EPISODES_DIR)
    batched_np_data = [np.array_split(datum, len(np_data[0]) // BATCH_SIZE) for datum in np_data]
    batched_mem = list(zip(*batched_np_data))

    # Optimize
    value_model.optimize_val_net(temp_policy_args, batched_mem, LEARNING_RATE, tb_metrics)
    # Resets the metrics
    metrics.reset_metrics(tb_metrics)
    
    # Evaluate against checkpoint model and other baselines
    rand_win_rate = data.make_episodes(temp_policy_args, random_policy_args, 
                                       NUM_EVAL_GAMES, num_workers=NUM_WORKERS)
    opp_win_rate = data.make_episodes(temp_policy_args, checkpoint_policy_args, 
                                      NUM_EVAL_GAMES, num_workers=NUM_WORKERS)

    stats = f"{100*opp_win_rate:.1f}%O, {100*rand_win_rate:.1f}%R"

    # If it's better than the checkpoint, update
    if opp_win_rate > 0.6:
        shutil.copy(TMP_WEIGHTS_PATH, CHECKPOINT_PATH)
        print(f"{stats} Accepted new model")
        
        greed_win_rate = data.make_episodes(temp_policy_args, greedy_policy_args, 
                                        NUM_EVAL_GAMES, num_workers=NUM_WORKERS)

        # Plot samples of states and response heatmaps
        fig = metrics.gen_traj_fig(go_env, temp_policy_args)
        fig.savefig(DEMO_TRAJECTORY_PATH)
        plt.close()

    elif opp_win_rate >= 0.5:
        print(f"{stats} Continuing to train current weights")

    else:
        shutil.copy(CHECKPOINT_PATH, TMP_WEIGHTS_PATH)
        print(f"{stats} Rejected new model")

values vs. values: 100%|██████████| 32/32 [04:52<00:00,  9.15s/it, 53.1% WIN]


53.1%O, 84.4%R Continuing to train current weights


values vs. values: 100%|██████████| 32/32 [04:56<00:00,  9.26s/it, 75.0% WIN]


75.0%O, 96.9%R Accepted new model


values vs. values: 100%|██████████| 32/32 [04:40<00:00,  8.77s/it, 59.4% WIN]


...

6.2%O, 100.0%R Rejected new model


values vs. values: 100%|██████████| 32/32 [05:06<00:00,  9.59s/it, 43.8% WIN]


43.8%O, 100.0%R Rejected new model


values vs. values:   0%|          | 0/128 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Evaluate

Play against our AI

In [27]:
human_policy_args = {
    'mode': 'human',
    'board_size': BOARD_SIZE,
}

In [29]:
val_net.load_weights(CHECKPOINT_PATH)

In [30]:
val_net.save('novice_9x9.h5')

In [28]:
data.make_episodes(checkpoint_policy_args, human_policy_args, 1, num_workers=1)

Episode worker: 0it [00:00, ?it/s]

    0   1   2   3   4   5   6   7   8
  -------------------------------------
0 | . | . | . | . | . | . | . | . | . |
  -------------------------------------
1 | . | . | . | . | . | . | . | . | . |
  -------------------------------------
2 | . | . | . | . | . | . | . | . | . |
  -------------------------------------
3 | . | . | . | . | . | . | . | . | . |
  -------------------------------------
4 | . | B | . | . | . | . | . | . | . |
  -------------------------------------
5 | . | . | . | . | . | . | . | . | . |
  -------------------------------------
6 | . | . | . | . | . | . | . | . | . |
  -------------------------------------
7 | . | . | . | . | . | . | . | . | . |
  -------------------------------------
8 | . | . | . | . | . | . | . | . | . |
  -------------------------------------
	Turn: W, Last Turn Passed: False, Game Over: 0
	Black Area: 81, White Area: 0

Enter coordinates separated by space (`q` to quit)
4 4
    0   1   2   3   4   5   6   7   8
  ---------------------------

Enter coordinates separated by space (`q` to quit)
4 6
    0   1   2   3   4   5   6   7   8
  -------------------------------------
0 | . | . | . | . | . | . | . | . | . |
  -------------------------------------
1 | . | . | B | . | . | . | . | . | . |
  -------------------------------------
2 | . | . | . | . | W | W | W | . | . |
  -------------------------------------
3 | . | . | . | W | B | B | B | W | . |
  -------------------------------------
4 | . | B | . | B | W | . | W | . | . |
  -------------------------------------
5 | . | . | . | B | W | W | . | . | . |
  -------------------------------------
6 | . | . | . | . | B | B | B | . | . |
  -------------------------------------
7 | . | . | . | . | . | . | . | . | . |
  -------------------------------------
8 | . | . | . | . | . | . | . | . | . |
  -------------------------------------
	Turn: W, Last Turn Passed: False, Game Over: 0
	Black Area: 10, White Area: 9

Enter coordinates separated by space (`q` to quit)
4 5
    0   1   

Enter coordinates separated by space (`q` to quit)
7 5
    0   1   2   3   4   5   6   7   8
  -------------------------------------
0 | . | . | . | . | . | . | . | . | . |
  -------------------------------------
1 | . | . | B | . | . | . | . | . | . |
  -------------------------------------
2 | . | . | B | . | W | W | W | . | . |
  -------------------------------------
3 | . | . | . | W | . | . | . | W | . |
  -------------------------------------
4 | . | B | B | B | W | W | W | . | . |
  -------------------------------------
5 | . | B | W | B | W | W | . | . | . |
  -------------------------------------
6 | . | B | W | W | B | B | B | B | . |
  -------------------------------------
7 | . | B | B | W | B | W | . | . | . |
  -------------------------------------
8 | . | . | W | W | . | . | . | . | . |
  -------------------------------------
	Turn: W, Last Turn Passed: False, Game Over: 0
	Black Area: 15, White Area: 20

Enter coordinates separated by space (`q` to quit)
8 4
    0   1  

Enter coordinates separated by space (`q` to quit)
1 4
    0   1   2   3   4   5   6   7   8
  -------------------------------------
0 | . | . | . | B | . | . | . | . | . |
  -------------------------------------
1 | . | . | B | W | W | B | B | B | . |
  -------------------------------------
2 | . | . | B | . | W | W | W | B | . |
  -------------------------------------
3 | . | . | B | W | . | . | . | W | . |
  -------------------------------------
4 | . | B | B | B | W | W | W | . | B |
  -------------------------------------
5 | . | B | W | B | W | W | W | W | W |
  -------------------------------------
6 | . | B | W | W | . | . | . | . | . |
  -------------------------------------
7 | . | B | B | W | . | W | W | . | W |
  -------------------------------------
8 | . | . | W | W | W | . | . | W | . |
  -------------------------------------
	Turn: W, Last Turn Passed: False, Game Over: 0
	Black Area: 17, White Area: 39

Enter coordinates separated by space (`q` to quit)
0 4
    0   1  

Enter coordinates separated by space (`q` to quit)
p
    0   1   2   3   4   5   6   7   8
  -------------------------------------
0 | . | . | B | B | W | B | . | . | . |
  -------------------------------------
1 | . | . | B | W | W | B | B | B | . |
  -------------------------------------
2 | . | . | B | B | W | W | W | B | W |
  -------------------------------------
3 | . | . | B | . | B | . | W | W | W |
  -------------------------------------
4 | . | B | B | B | W | W | W | W | . |
  -------------------------------------
5 | . | B | W | B | W | W | W | W | W |
  -------------------------------------
6 | . | B | W | W | . | . | . | B | . |
  -------------------------------------
7 | . | B | B | W | . | W | W | . | W |
  -------------------------------------
8 | . | W | W | W | W | . | . | W | . |
  -------------------------------------
	Turn: W, Last Turn Passed: False, Game Over: 0
	Black Area: 22, White Area: 35

Enter coordinates separated by space (`q` to quit)
p
    0   1   2  

Episode worker: 1it [04:00, 240.51s/it]

values vs. human: 100%|██████████| 1/1 [00:00<00:00, 450.23it/s, 100.0% WIN]


1.0